In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Input, LayerNormalization, MultiHeadAttention,GlobalAveragePooling1D,GRU


In [ ]:
df = pd.read_csv("/content/drive/My Drive/final_train_df.csv")
test_df = pd.read_csv("/content/drive/My Drive/final_test_df.csv")
val_df = pd.read_csv("/content/drive/My Drive/final_val_df.csv")

df.drop(columns=['Unnamed: 0', 'User_ID'], inplace=True)
test_df.drop(columns=['Unnamed: 0', 'User_ID'], inplace=True, errors='ignore')
val_df.drop(columns=['Unnamed: 0', 'User_ID'], inplace=True, errors='ignore')

In [ ]:
df.head()

,Age,Gender,Platform,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day,Dominant_Emotion
0,25,Female,Instagram,120.0,3.0,45.0,10.0,12.0,Happiness
1,30,Male,Twitter,90.0,5.0,20.0,25.0,30.0,Anger
2,22,Non-binary,Facebook,60.0,2.0,15.0,5.0,20.0,Neutral
3,28,Female,Instagram,200.0,8.0,100.0,30.0,50.0,Anxiety
4,33,Male,LinkedIn,45.0,1.0,5.0,2.0,10.0,Boredom


In [ ]:
encoder = OneHotEncoder(sparse_output = False, drop='first')
encoder_categories = encoder.fit_transform(df[['Gender', 'Platform']])
encoder_df = pd.DataFrame(encoder_categories,columns=encoder.get_feature_names_out())

In [ ]:
encoded_cats_test = encoder.fit_transform(test_df[['Gender', 'Platform']])
encoded_cats_val = encoder.transform(val_df[['Gender', 'Platform']])

In [ ]:
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(df[['Age', 'Daily_Usage_Time (minutes)',
                                            'Posts_Per_Day', 'Likes_Received_Per_Day',
                                            'Comments_Received_Per_Day', 'Messages_Sent_Per_Day']])

scaled_df = pd.DataFrame(scaled_numerical, columns=['Age', 'Daily_Usage_Time (minutes)',
                                                    'Posts_Per_Day', 'Likes_Received_Per_Day',
                                                    'Comments_Received_Per_Day', 'Messages_Sent_Per_Day'])


In [ ]:
scaled_numerical_test = scaler.fit_transform(test_df[['Age', 'Daily_Usage_Time (minutes)',
                                                      'Posts_Per_Day', 'Likes_Received_Per_Day',
                                                      'Comments_Received_Per_Day', 'Messages_Sent_Per_Day']])

scaled_numerical_val = scaler.transform(val_df[['Age', 'Daily_Usage_Time (minutes)',
                                                'Posts_Per_Day', 'Likes_Received_Per_Day',
                                                'Comments_Received_Per_Day', 'Messages_Sent_Per_Day']])


In [ ]:
X_test = np.hstack((scaled_numerical_test, encoded_cats_test))
X_val = np.hstack((scaled_numerical_val, encoded_cats_val))

In [ ]:
y_test = pd.get_dummies(test_df['Dominant_Emotion']).values
y_val = pd.get_dummies(val_df['Dominant_Emotion']).values

In [ ]:
X = pd.concat([scaled_df, encoder_df], axis=1)
y = pd.get_dummies(df['Dominant_Emotion'])

In [ ]:
print(X.shape)
print(y.shape)

(1000, 14)
(1000, 6)


In [ ]:
# model = Sequential([
#     Dense(64, activation='relu', input_shape=(X.shape[1],)),
#     Dropout(0.2),
#     Dense(32, activation='relu'),
#     Dropout(0.2),
#     Dense(y.shape[1], activation='softmax')
# ])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(X, y, epochs=80, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.2220 - loss: 1.7902 - val_accuracy: 0.4500 - val_loss: 1.5460
Epoch 2/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3803 - loss: 1.5740 - val_accuracy: 0.5000 - val_loss: 1.4062
Epoch 3/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4388 - loss: 1.4615 - val_accuracy: 0.5000 - val_loss: 1.3179
Epoch 4/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4745 - loss: 1.3654 - val_accuracy: 0.5300 - val_loss: 1.2499
Epoch 5/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5141 - loss: 1.3084 - val_accuracy: 0.5500 - val_loss: 1.1971
Epoch 6/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5371 - loss: 1.2242 - val_accuracy: 0.5700 - val_loss: 1.1567
Epoch 7/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5045 - loss: 1.2457 - val_accuracy: 0.6300 - val_loss: 1.1138
Epoch 8/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5474 - loss: 1.1876 - val_accuracy: 0.6400 - val_loss

In [ ]:
import numpy as np
X_reshaped = np.expand_dims(X.values, axis=1)
print("reshaped",X_reshaped.shape)

reshaped (1000, 1, 14)


In [ ]:
gru_model = Sequential([
    GRU(64, return_sequences=True, input_shape=(1, X_reshaped.shape[2])),
    Dropout(0.2),
    GRU(32, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(y.shape[1], activation='softmax')  # Multi-class output
])
gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the GRU model
gru_model.fit(X_reshaped, y, epochs=150, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/150
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.3226 - loss: 1.7412 - val_accuracy: 0.4300 - val_loss: 1.6184
Epoch 2/150
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4301 - loss: 1.5737 - val_accuracy: 0.4300 - val_loss: 1.4588
Epoch 3/150
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4083 - loss: 1.4767 - val_accuracy: 0.4600 - val_loss: 1.3930
Epoch 4/150
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4581 - loss: 1.4014 - val_accuracy: 0.5000 - val_loss: 1.3393
Epoch 5/150
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4869 - loss: 1.3028 - val_accuracy: 0.4900 - val_loss: 1.2949
Epoch 6/150
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4713 - loss: 1.3307 - val_accuracy: 0.5200 - val_loss: 1.2587
Epoch 7/150
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5162 - loss: 1.2496 - val_accuracy: 0.4900 - val_loss: 1.2259
Epoch 8/150
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5259 - loss: 1.2518 - val_accuracy: 0.5400 

In [ ]:
gru_model.save("social_media_emotion_gru.keras")

In [ ]:
gru_model = load_model("social_media_emotion_gru.keras")

In [ ]:
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
print("Expected input shape by model:", gru_model.input_shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)


Expected input shape by model: (None, 1, 14)
X_val shape: (144, 1, 15)
X_test shape: (103, 1, 15)


In [ ]:
print("Feature count in X_val:", X_val.shape[-1])
print("Feature count in trained model:", gru_model.input_shape[-1])


Feature count in X_val: 15
Feature count in trained model: 14


In [ ]:
X_val = X_val[:, :, :gru_model.input_shape[-1]]
X_test = X_test[:, :, :gru_model.input_shape[-1]]

In [ ]:
X_val.shape

(144, 1, 14)

In [ ]:
X_test.shape

(103, 1, 14)

In [ ]:
y_val.shape

(144, 7)

In [ ]:
y_val = y_val[:, :gru_model.output_shape[1]]
y_test = y_test[:, :gru_model.output_shape[1]]


In [ ]:
y_val.shape

(144, 6)

In [ ]:
y_test.shape

(103, 6)

In [ ]:
val_loss, val_acc = gru_model.evaluate(X_val, y_val, verbose=1)
print(f"Validation Accuracy: {val_acc:.4f}")

test_loss, test_acc = gru_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_acc:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0647 - loss: 11.3757   
Validation Accuracy: 0.0833
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4638 - loss: 4.3034
Test Accuracy: 0.4563


In [ ]:
# Make predictions
y_pred_probs = gru_model.predict(X_test)
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Convert one-hot encoded test labels back to category
y_test_labels = np.argmax(y_test, axis=1)

# Compare the first 10 predictions
for i in range(10):
    print(f"Actual: {y_test_labels[i]}, Predicted: {y_pred_labels[i]}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 445ms/step
Actual: 4, Predicted: 3
Actual: 4, Predicted: 4
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 4, Predicted: 4
Actual: 3, Predicted: 3
Actual: 3, Predicted: 3
Actual: 4, Predicted: 4
Actual: 3, Predicted: 4
Actual: 2, Predicted: 5
